In [1]:
:opt no-lint

# Tautology checker (주교재 8.6절)

In [2]:
type Nm = String -- 책에서는 Char이지만 우리는 String으로 하자

data Prop = Const Bool      -- True, False
          | Var Nm          -- A,B,C
          | Not Prop        -- ¬ P
          | And Prop Prop   -- P1 ∧ P2
          | Imply Prop Prop -- P1 ⇒ P2
          deriving Show

In [3]:
vA = Var "A"
vB = Var "B"
vC = Var "C"

p1 = And vA (Not vB) -- -- A /\ ㄱB
p2 = Imply vA vB -- A => B
p3 = Imply (And vA vB) vA -- (A /\ B) => A
p4 = Imply (And vA (Imply vA vB)) vB -- (A /\ (A=>B)) => B
p5 = Imply (And (Imply vA vB) (Imply vB vC)) (Imply vA vC)
     -- ((A=>B)/\(B=>C)) => (A=>C)

In [4]:
type Subst = [ (Nm, Bool) ] -- 덧셈식이나 람다식에서 Env에 해당

eval :: Subst -> Prop -> Bool
eval _ (Const b)   = b
eval s (Var x)     = find x s
eval s (Not p)     = not (eval s p)
eval s (And p q)   = eval s p && eval s q
eval s (Imply p q) = not(eval s p) || eval s q
          -- 책에서는 eval s p <= eval s q

find x s = case lookup x s of
             Nothing -> error $ x++" not defined"
             Just b -> b

In [5]:
vA

Var "A"

In [6]:
eval [("A",False)] (And vA (Const True))
eval [("A",True)] (And vA (Const True))

False

True

In [7]:
eval [("A",False)] (Imply (Const False) vA)
eval [("A",True)] (Imply (Const False) vA)

True

True

주교재에서는 책의 앞부분에 정의한 `rmdup`라는 함수를 이용해
나중에 `vars` 함수의 결과로부터 중복을 제거하지만,
우리는 그냥 여기서 `++` 대신 `union`을 써서 `vars`를 정의함으로써
중복된 변수 이름에 대한 걱정을 덜어버리자.

In [8]:
import Data.List -- union 등

vars :: Prop -> [Nm]
vars (Const _)   = []
vars (Var x)     = [x]
vars (Not p)     = vars p
vars (And p q)   = vars p `union` vars q
vars (Imply p q) = vars p `union` vars q

In [9]:
["A","B"] ++ ["B","C"]
["A","B"] `union` ["B","C"]

["A","B","B","C"]

["A","B","C"]

In [10]:
vars p4
vars p5

["A","B"]

["A","B","C"]

In [11]:
map (2*) [1,2,3,4]

[2,4,6,8]

In [12]:
3+10
f = (3+)
f 10

13

13

In [13]:
True:[False,True]
g = (True:)
g [False,True]

[True,False,True]

[True,False,True]

In [14]:
f0 = \bs -> False:bs
f1 = \bs -> True:bs

f0 [False]
f1 [False]

[False,False]

[True,False]

In [15]:
[ [False], [True] ] -- bools 1

-- bools (1+1)
map (False:) [ [False], [True] ] ++ map f1 [ [False], [True] ]

[[False],[True]]

[[False,False],[False,True],[True,False],[True,True]]

In [16]:
bools :: Int -> [[Bool]]
bools n | n <= 0    = [[]]
        | otherwise = map (False:) bss ++ map (True:) bss
        -- | otherwise = [b:bs | b<-[False,True], bs<-bss]
        where bss = bools (n-1)

In [17]:
bools 0

[[]]

In [18]:
bools 1

[[False],[True]]

In [19]:
bools 2
putStrLn "-----"
mapM_ print $ bools 2

[[False,False],[False,True],[True,False],[True,True]]

-----

[False,False]
[False,True]
[True,False]
[True,True]

In [20]:
bools 3
putStrLn "-----"
mapM_ print $ bools 3

[[False,False,False],[False,False,True],[False,True,False],[False,True,True],[True,False,False],[True,False,True],[True,True,False],[True,True,True]]

-----

[False,False,False]
[False,False,True]
[False,True,False]
[False,True,True]
[True,False,False]
[True,False,True]
[True,True,False]
[True,True,True]

In [21]:
(zip ["A","B"]) [False,False]
length ["A","B"]

[("A",False),("B",False)]

2

In [22]:
(zip ["A","B"]) [False,False]
(zip ["A","B"]) [False,True] 
(zip ["A","B"]) [True, False]
(zip ["A","B"]) [True, True] 

map (zip ["A","B"]) [ [False,False]
                    , [False,True] 
                    , [True, False]
                    , [True, True]  ]

[("A",False),("B",False)]

[("A",False),("B",True)]

[("A",True),("B",False)]

[("A",True),("B",True)]

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

In [23]:
(zip ["A","B","C"]) [False,False,True]
length ["A","B","C"]

[("A",False),("B",False),("C",True)]

3

In [24]:
substs :: Prop -> [Subst]
substs p = map (zip vs) (bools (length vs))
        -- [zip vs bs | bs<-bools (length vs)]
        where vs = vars p

In [25]:
pp1 = Imply vA (And vA vB) -- A => (A /\ B)
pp2 = Imply (And vA vB) vA -- (A /\ B) => A

In [26]:
substs pp1
{-
eval [("A",False),("B",False)] pp1
eval [("A",False),("B",True)] pp1
eval [("A",True),("B",False)] pp1
eval [("A",True),("B",True)] pp1
-}
map (\s -> eval s pp1) (substs pp1)

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

[True,True,False,True]

In [27]:
substs pp2

eval [("A",False),("B",False)] pp2
eval [("A",False),("B",True)] pp2
eval [("A",True),("B",False)] pp2
eval [("A",True),("B",True)] pp2

map (\s -> eval s pp2) (substs pp2)

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

True

True

True

True

[True,True,True,True]

In [28]:
substs p4
vars p4
putStrLn "-----"
mapM_ print $ substs p4

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

["A","B"]

-----

[("A",False),("B",False)]
[("A",False),("B",True)]
[("A",True),("B",False)]
[("A",True),("B",True)]

In [29]:
substs p5
vars p5
putStrLn "-----"
mapM_ print $ substs p5

[[("A",False),("B",False),("C",False)],[("A",False),("B",False),("C",True)],[("A",False),("B",True),("C",False)],[("A",False),("B",True),("C",True)],[("A",True),("B",False),("C",False)],[("A",True),("B",False),("C",True)],[("A",True),("B",True),("C",False)],[("A",True),("B",True),("C",True)]]

["A","B","C"]

-----

[("A",False),("B",False),("C",False)]
[("A",False),("B",False),("C",True)]
[("A",False),("B",True),("C",False)]
[("A",False),("B",True),("C",True)]
[("A",True),("B",False),("C",False)]
[("A",True),("B",False),("C",True)]
[("A",True),("B",True),("C",False)]
[("A",True),("B",True),("C",True)]

In [30]:
and [True,True,True] -- && 리스트 버전
and [True,False,True]

True

False

In [31]:
or [False,False,False] -- || 리스트 버전
or [False,True,False]

False

True

In [32]:
isTaut :: Prop -> Bool
isTaut p = and (map (\s -> eval s p) (substs p))
         -- and [eval s p | s <- substs p]

In [33]:
-- 만족가능한(satisfiable) 명제인지 검사 
isSat :: Prop -> Bool
isSat p = or (map (\s -> eval s p) (substs p))

unSat = not . isSat

In [34]:
pp1
isTaut pp1

Imply (Var "A") (And (Var "A") (Var "B"))

False

In [35]:
pp2
isTaut pp2

Imply (And (Var "A") (Var "B")) (Var "A")

True

In [36]:
pp3 = And vA (Not vA) -- 만족가능하지 않음
map (\s -> eval s pp3) (substs pp3)

[False,False]

In [37]:
print p1 -- A /\ ㄱB
isTaut p1
print p2 -- A => B
isTaut p2
print p3 -- (A /\ B) => A
isTaut p3
print p4 -- (A /\ (A=>B)) => B
isTaut p4
print p5 -- ((A=>B)/\(B=>C)) => (A=>C)
isTaut p5

And (Var "A") (Not (Var "B"))

False

Imply (Var "A") (Var "B")

False

Imply (And (Var "A") (Var "B")) (Var "A")

True

Imply (And (Var "A") (Imply (Var "A") (Var "B"))) (Var "B")

True

Imply (And (Imply (Var "A") (Var "B")) (Imply (Var "B") (Var "C"))) (Imply (Var "A") (Var "C"))

True